In [1]:
import numpy as np

def pdrv(g,x,j,h):
    y=np.empty(len(x))
    for i in range(len(x)):
        if i==j:
            y[i]=x[i]+h
        else:
            y[i]=x[i]
            
    return (g(y)-g(x))/(h)

def grad(g,x,h):
    y=np.empty(len(x))
    for i in range(len(x)):
        y[i]=pdrv(g,x,i,h)
        
    return y

def gradientdescent(f, x, method,tol=1e-6, maxit=10000):
    
    if method=="constant":
        a=0.001
        iter=0
        while(True):
            d=-grad(f,x,1e-6)
            if(np.linalg.norm(d)<tol or iter>maxit):
                break
            x=x+a*d
            iter+=1
        
        return x
    
    elif method=="armijo":
        
        p=0.5
        c1=0.0001
        t=0.02
        iter=0
        
        while(True):
            t=p*t
            d=-grad(f,x,1e-6)
            x_p=f(x)
            x=x+t*d
            if(f(x)>x_p+c1*t*np.dot(-d,d)or iter>maxit):
                break
            iter+=1
        return x
    
    elif method=="wolfe":
        
        p=0.5
        c1=0.0001
        c2=0.9
        t=0.02
        iter=0
        
        while(True):
            t=p*t
            d=-grad(f,x,1e-6)
            x_p=f(x)
            x=x+t*d
            if((f(x)>x_p+c1*t*np.dot(-d,d)) or (np.dot(grad(f,x,1e-6),d)<c2*np.dot(-d,d)) or iter<maxit):
                break
            iter+=1
            
        return x  

In [2]:
class CallBack:
    """Call back
    
    Collects information aboute the iterates xk in a list 
    self.xk.
    """
    def _init_(self):
        # The Class constructor is executed when 
        # we create an new isntance by obj = CallBack().
        # It takes no arguments.
        self.xk=[]
        
    def _call_(self, xk):
        # The means an object obj = CallBack
        # can be executed like a function by obj().
        self.xk.append(xk.copy())
        return False
    def getxk(self):
        return np.array(self.xk)
    
    def plot(self, f, xmin=-1.5, xmax=1.5):
        
        Xk = np.array(self.xk)
        l = np.arange(xmin,xmax,.01)
        X,Y = np.meshgrid(l,l)
        XY = np.vstack([X.ravel(),Y.ravel()]).T
        Z = np.array([f(xy) for xy in XY])
        Z=Z.reshape(X.shape)
        plt.contourf(X,Y,Z,levels=15)
        plt.contour(X,Y,Z,levels=10)
        zk = np.array([f(xk) for xk in self.xk])
        plt.scatter(Xk[:,0], Xk[:,1], c="white", s=15)
        plt.plot(Xk[:,0], Xk[:,1], color="white", alpha=.6)
        #plt.scatter(0,0,c="r")
        plt.show()

Q = np.array([[5, 1],[1, 50]])
def f(x):
    return x@Q@x

def test(gradientdescent):
    x0 = np.ones(2)
    method = ["constant", "armijo", "wolfe"]
    
    for m in method:
        try:
            solution = gradientdescent(f, x0, method=m )
            print("Method: ", m, "\tx = ", np.round(solution, 4))
        except:
            print("Method", m, "does not work.")
            raise

In [3]:
test(gradientdescent)

Method:  constant 	x =  [-0. -0.]
Method:  armijo 	x =  [ 0.7953 -0.0178]
Method:  wolfe 	x =  [ 0.88 -0.02]
